# HW1 Overview

In this assignment, you will get familiar with basic document representation and analysis techniques. You will get the basic ideas of tokenization, stemming, normalization, constructing bag of words representation for text documents, and building an inverted index for efficient access.



# Data Set

The instructor has prepared a small size collection of Yelp restaurant reviews (included in the provided sample Java project and separated into three folders) for this assignment. The review files are named and organized in the following manner:

1. Each file contains all crawled review documents for a specific business on Yelp, and it is named by its unique ID on Yelp, e.g., *FAhx3UZtXvqNiEAd-GNruQ.json*;
2. All the files are in json format. Each json file contains a json array of reviews ('Reviews') and a json object about the information of the restaurant ('RestaurantInfo').              

	2.1 The json object for **user review** is defined as follows:           
		{          
			'Author':'author name (string)',
			'ReviewID':'unique review id (string)',  
			'Overall':'numerical rating of the review (float)',
			'Content':'review text content (string)',   
			'Date':'date when the review was published',   
			'Author_Location':'author's registered location'  
		} 
    
	2.2 The json object for **restaurant info** is defined as follows:                                
		{                
			'RestaurantID':'unique business id in Yelp (string)',    
			'Name':'name of the business (string)',      
			'Price':'Yelp defined price range (string)',    
			'RestaurantURL':'actual URL to the business page on Yelp (string)',   
			'Longitude':'longitude of the business (double)',              
			'Latitude':'latitude of the business (double)',              
			'Address':'address of the business (string)',       
			'ImgURL':'URL to the business's image on Yelp (string)'     
		} 

In the following discussion, we will refer to each individual user review as a **document**. 
Note that some collected json files might not strictly follow the above json object definitions, e.g., some fields are missing or empty. As a result, properly handling the exceptions in json parsing is necessary.

In [ ]:
import os
import json
from nltk.tokenize import word_tokenize
import nltk
from string import punctuation
from nltk.stem import PorterStemmer
import re
import sys
import matplotlib.pyplot as plt
from elasticsearch import Elasticsearch
import time
def is_integer(s):
    try:
        int(s)
        return True
    except ValueError:
        return False


def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


def is_fraction(s):
    fraction_pattern = re.compile(r'^\d+/\d+$')
    return bool(fraction_pattern.match(s))


def remove_non_alphanumeric(input_string):
    # Use regular expression to keep only English letters and numbers
    result = re.sub(r'[^a-zA-Z0-9]', '', input_string)
    return result

def is_stop_word(token):
    stop_words = set([
        'a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in',
        'into', 'is', 'it', 'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the',
        'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will', 'with'
    ])

    # Convert the word to lowercase for case-insensitive comparison
    lowercase_word = token.lower()

    return lowercase_word in stop_words



# Basic Text Processing

Apply NLTK library (http://www.nltk.org/install.html). Recall the following steps to pre-process the document: tokenization, normalization, stemming, stopwords removal.

- Tokenization: tokenize the review content of each document into tokens.
- Normalization: normalize the tokens from step 1, by removing individual punctuation marks (here is a list of [English punctuation marks](http://en.wikipedia.org/wiki/Punctuation_of_English)), converting tokens into lower cases, and recognizing digit numbers, e.g., integers and floats, and map them to a special symbol "NUM". 
- Stemming: stem the tokens back to their root form. 

## 1. Understand Zipf's Law (50pts)

First, let's validate Zipf's law with the provided Yelp review data sets. This can be achieved by the following steps:

1. Process the text document according to the discussed steps above.
2. For each token, go over all the review documents containing it, and accumulate its frequency in those documents, i.e., total term frequency (TTF).
3. Order the tokens by their TTF in descending order.
4. Create a dot plot by treating each word's rank as x-axis and its TTF as y-axis. Please use log-log scale for this plot.

*Hint: basically, you can maintain a look-up table in memory while you are scanning through the documents, so that you only need to go through the corpus once to get the counts for all the tokens. In the provided implementation, this look-up table is maintained in the "Corpus" class, named "m_dictionary".*

From the resulting plot, can we find a strong linear relationship between the x and y axes in the log-log scale? If so, what is the slope of this linear relationship? To answer these questions, you can dump the data into Excel and use the plot function there, or use some scientific computing packages in python for curve fitting for this purpose.         

Then change the counting statistics in the previous experiment from *total term frequency (TTF)* to *document frequency (DF)* (i.e., how many documents contain this specific term), and perform the experiment again. According to new curve and corresponding slope and intercept of the linear interpretation, can you conclude which counting statistics, i.e., *TTF* v.s., *DF*, fits Zipf's law better on this data set? Can you give any explanation about why it fits the law better?

**What to submit**: 

1. Paste your implementation of text normalization module. (15pts)
2. Two curves in log-log space generated above, with the corresponding slopes and intercepts of the linear interpretation results. (20pts)
3. Your answers and thoughts to the above questions. (15pts)


## 2. Building An Inverted Index (50pts)

An inverted index accelerates text corpus access by building a term to document mapping. In this task, we will compare efficiency speed-up provided by such a specialized data structure. We will apply Elasticsearch(Python) - https://pypi.python.org/pypi/elasticsearch to build the index and query it. Check the [docs](https://elasticsearch-py.readthedocs.io/en/v8.12.0/) and these [examples](https://www.elastic.co/guide/en/elasticsearch/client/python-api/master/examples.html#ex-index). To query a document, we can first use ES to query with the API, such as 
> "match": {"\<filed\>":"query keywords"}. 

This should be re with its default BM25 retrieval model. Then implement Bag-of-Word retrieval model without using ES query.

Use these two retrieval methods, i.e., inverted index and brute-force search over your BoW document representation, to count the total number of documents that match with the 10 queries listed below accordingly,

	general chicken
	fried chicken
	BBQ sandwiches
	mashed potatoes
	Grilled Shrimp Salad
	lamb Shank
	Pepperoni pizza
	brussel sprout salad
	FRIENDLY STAFF
	Grilled Cheese

Please record the total running time of each of retrieval methods, and the number of returned documents accordingly.

**What to submit**: 

1. Paste your implementation of Bag-of-Word document representation (e.g., how to construct it from raw document content, and how to search for a particular term in it). (15pts)
2. Two curves in log-log space generated by using Elasticsearch inverted index for collecting TTF and DF statistics, with the corresponding running time statistics in comparison with your implementation in problem one. (15pts)
3. Running time and total number of returned documents by two retrieval models. If you found these two methods gave you different number of matched documents, do you have any explanation about it? (20pts)


# Extra Credits (10pts)

You are encouraged to further investigate Zipf's law. For example (but not limited to), subsample the reviews with different sizes and see if an increased number of documents better help us verify Zipf's law by comparing the log-log curves.


# Submission

This assignment has in total 100 points. The deadline is Feb 6 23:59 PDT. You should submit your report in **PDF** using the homework latex template, and submit your code (notebook). 